In [1]:
from tqdm import tqdm
import numpy as np
import os

import cv2
import albumentations as A

import torch
import torch.nn as nn
import torch.optim as optim
from torch.nn import BCEWithLogitsLoss
import torch.nn.functional as F
from torch.cuda.amp import GradScaler, autocast
import segmentation_models_pytorch as smp
from torchvision.utils import make_grid, save_image

import model
import metrics
import config
import utis

import matplotlib.pyplot as plt

from torch.utils.data import DataLoader, random_split
from dataset import HistoricalImagesDataset
import config
import glob

# Ignore excessive warnings
import logging
logging.propagate = False 
logging.getLogger().setLevel(logging.ERROR)

# WandB – Import the wandb library
import wandb

import albumentations as A
import albumentations.augmentations.functional as F
from albumentations.pytorch import ToTensorV2

In [2]:
# Paths
IMAGES_PATH = '../data/patches/images/1942'
MASK_PATH = '../data/patches/masks/1942'
BEST_MODEL = '../best_model'
# PREDICTIONS_PATH = 

image_paths = glob.glob(IMAGES_PATH +'/*.tif')[:100]
mask_paths = glob.glob(MASK_PATH +'/*.tif')[:100]

In [3]:
cv2.imread(image_paths[0], cv2.IMREAD_GRAYSCALE)

array([[ 95, 117, 118, ..., 213, 212, 203],
       [102, 120, 122, ..., 212, 212, 211],
       [104, 117, 117, ..., 209, 212, 216],
       ...,
       [162, 176, 178, ..., 120, 119, 118],
       [159, 167, 168, ..., 129, 115, 112],
       [186, 192, 187, ..., 131, 129, 135]], dtype=uint8)

In [4]:
from PIL import Image

image = Image.open(image_paths[0])
mask = Image.open(mask_paths[0])
image = np.array(image)
mask = np.array(mask)
 
transform = A.Compose([
      A.Rotate(limit=40,p=0.9, border_mode=cv2.BORDER_CONSTANT), # p stands for the probability with which the transformations are applied
      A.HorizontalFlip(p=0.5),
      A.VerticalFlip(p=0.1), 
      A.Normalize(mean=(0), std=(1)),
      ToTensorV2()])

# list_images = []
# for i in range(15):
#     augmentations = transform(image=image, mask=mask)
#     augmented_img = augmentations['image']
#     augmented_mask = augmentations['mask']

#     list_images.append(augmented_img)
#     list_images.append(augmented_mask)

In [5]:
# list_images


In [6]:
# plt.imshow(list_images[0][0])


### Weights and Biases

In [7]:
!wandb login

wandb: Currently logged in as: vibferreira (use `wandb login --relogin` to force relogin)


### Dataloader

In [8]:
# Dataset Object 
print('Number of image patches:', len(image_paths),'\nNumber of mask patches:', len(mask_paths))
dataset = HistoricalImagesDataset(image_paths, mask_paths, transform=transform)
data = next(iter(dataset))
print('shape image', data[0].shape, 'shape mask', data[1].shape)       

# Train, Test, Split -- DEVEOLP A SPLITTING STRATEGY BASED ON THE SPATIAL INFORMATION !!!!!!!!!
print('Splitting data into TRAIN, VAL and TEST')
train_size = int(0.5 * len(dataset))
test_size = len(dataset) - train_size
train_dataset, test_dataset = random_split(dataset, [train_size, test_size]) #-- pytorch alternative to the train_test_split command line from Scikit-Learn

train_size = int(0.5 * len(train_dataset))
val_size = len(train_dataset) - train_size
test_dataset, val_dataset = random_split(train_dataset, [train_size, val_size])

# DataLoader
print("Training set size: ", len(train_dataset))
train_dataloader = DataLoader(dataset=train_dataset, batch_size = config.BATCH_SIZE)
print("Validation set size: ", len(val_dataset))
val_dataloader = DataLoader(dataset=val_dataset, batch_size = config.BATCH_SIZE)
print("Testing set size: ", len(test_dataset))
test_dataloader = DataLoader(dataset=test_dataset, batch_size = config.BATCH_SIZE)

Number of image patches: 100 
Number of mask patches: 100
shape image torch.Size([1, 256, 256]) shape mask torch.Size([256, 256])
Splitting data into TRAIN, VAL and TEST
Training set size:  50
Validation set size:  25
Testing set size:  25


In [9]:
val_dataloader_iter = iter(val_dataloader)
# grids = utis.create_segement_grids(val_dataloader_iter)
# utis.plot_grids(grids)

In [10]:
next(val_dataloader_iter)[0].shape

torch.Size([8, 1, 256, 256])

### Check if CUDA is available

In [11]:
config.DEVICE

'cuda'

In [12]:
iter_n = len(train_dataset)/len(train_dataloader)
iter_n

7.142857142857143

### Training

In [13]:
def train(model, dataloader, optim, lossFunc, epoch, scaler):
    # set the model in training mode
    model.train()

    # Save total train loss
    totalTrainLoss = 0
    
    # metrics
    accuracy = 0
    iou = 0
    f1score = 0
    dice = 0
    
    # testing
    tp = 0
    fp = 0
    fn = 0
    tn = 0
    
    # loop over the training set
    loop = tqdm(dataloader, leave=False)
    for x, y in loop:
        # send the input to the device
        (x, y) = (x.to(config.DEVICE), y.float().to(config.DEVICE))
        
        # forward with autocast        
        with autocast():
            pred = model(x)
            loss = lossFunc(pred, y)
            
        optim.zero_grad()  # zero out any previously accumulated gradients    
        scaler.scale(loss).backward() # study this 
        scaler.step(optim)
        scaler.update()
        
        totalTrainLoss += loss  # add the loss to the total training loss so far 
        
        # metrics      
        all_metrics = metrics.metrics(pred, y)
        accuracy += all_metrics['acc']
        iou += all_metrics['iou']
        f1score += all_metrics['f1score']
        dice += all_metrics['dice_coeff']
        
        tp += all_metrics['false_preds'][0]
        fp += all_metrics['false_preds'][1]
        fn += all_metrics['false_preds'][2]
        tn += all_metrics['false_preds'][3]
        
        # update tqdm
        loop.set_description(f'Training Epoch [{epoch}/{config.NUM_EPOCHS}]')
        loop.set_postfix(loss=loss.item(), acc = all_metrics['acc'], iou=all_metrics['iou'], dice = all_metrics['dice_coeff'])
        
    # calculate the average training loss PER EPOCH
    avgTrainLoss = totalTrainLoss / len(dataloader)
    avgAccLoss = accuracy / len(dataloader)
    avgIOU = iou / iter_n
    avgF1score = f1score / len(dataloader)
    avgDice = dice / len(dataloader)
    
    avg_tp = tp / len(dataloader)
    avg_fp = fp / len(dataloader)
    avg_fn = fn / len(dataloader)
    avg_tn = tn / len(dataloader)
    
    ## update training history
    training_history["avg_train_loss"].append(avgTrainLoss.cpu().detach().numpy()) # save the avg loss
    training_history["train_accuracy"].append(avgAccLoss) # save the acc 
    
    # WANDB
    wandb.log({
    # "Examples": example_images,
    "true_positives": avg_tp, 
    "true_negatives": avg_tn, 
    "false_negatives": avg_fn,
    "false_positives": avg_fp,
    "Train Loss": avgTrainLoss,
    "Train Accuracy": avgAccLoss,
    "IoU_train":avgIOU})
    
    return training_history

In [14]:
def validation(model, dataloader, lossFunc, epoch):
    
    # set the model in evaluation mode
    model.eval()
    # Save total train loss
    totalValLoss = 0
    
    # metrics
    accuracy_val = 0
    iou_val = 0
    f1score_val = 0
     
    # testing
    tp = 0
    fp = 0
    fn = 0
    tn = 0
    
    # switch off autograd
    example_pred = []
    example_gt = []
    
    with torch.no_grad():
        # loop over the validation set
        loop = tqdm(dataloader, leave=False)
        for (x_val, y_val) in loop:
            # send the input to the device
            (x_val, y_val) = (x_val.to(config.DEVICE), y_val.to(config.DEVICE))
            
            # make the predictions and calculate the validation loss
            pred_val = model(x_val)
            loss = lossFunc(pred_val, y_val)
            totalValLoss += loss
            
            # metrics      
            all_metrics = metrics.metrics(pred_val, y_val)
            accuracy_val += all_metrics['acc']
            iou_val += all_metrics['iou']
            f1score_val += all_metrics['f1score']
            
            tp += all_metrics['false_preds'][0]
            fp += all_metrics['false_preds'][1]
            fn += all_metrics['false_preds'][2]
            tn += all_metrics['false_preds'][3]
            
            # # Plotting Val 
            # if iter_  % 2 == 0: # plot every time iter is a multiple of 2
            # utis.plot_comparison(x_val, pred_val, y_val)
            # iter_ + 1
            print(y_val[0].shape)
            print(type(y_val[0]))
            # WandB – Log images in your test dataset automatically, along with predicted and true labels by passing pytorch tensors with image data into wandb.Image
            example_pred.append(wandb.Image(pred_val[0], caption="pred"))
            # example_gt.append(wandb.Image(y_val[0], caption="GT"))
            
            # update tqdm
            loop.set_description(f'Validation Epoch [{epoch}/{config.NUM_EPOCHS}]')
            loop.set_postfix(loss_val=loss.item(), acc_val = all_metrics['acc'], iou_val=all_metrics['iou'])
                        
    # calculate the average VALIDATION loss PER EPOCH
    avgValLoss = totalValLoss / len(dataloader)
    avgAccLoss = accuracy_val / len(dataloader)
    avgIOU = iou_val / len(dataloader)
    avgF1score = f1score_val / len(dataloader)
    
    avg_tp = tp / len(dataloader)
    avg_fp = fp / len(dataloader)
    avg_fn = fn / len(dataloader)
    avg_tn = tn / len(dataloader)

    ## update VALIDATION history
    validation_history["avg_val_loss"].append(avgValLoss.cpu().detach().numpy()) # save the avg loss
    validation_history["val_accuracy"].append(avgAccLoss) # save the acc
    
    # WANDB
    wandb.log({
    "Predictions": example_pred,
    # "GT": example_gt,
    "true_positives_val": avg_tp, 
    "true_negatives_val": avg_tn, 
    "false_negatives_val": avg_fn,
    "false_positives_val": avg_fp,
    "Val Accuracy": avgAccLoss,
    "Val Loss": avgValLoss,
    "IoU_val": avgIOU})
    
    return validation_history

### MAIN

In [15]:
# WandB – Initialize a new run
wandb.init(entity="vibferreira", project="master_thesis")

# classes
classes = ('no_vegetation', 'vegetation')

# Initialize model
unet = model.unet_model.to(config.DEVICE)

# initialize loss function and optimizer
lossFunc = smp.losses.DiceLoss(smp.losses.BINARY_MODE, from_logits=True)
opt = optim.Adam(unet.parameters(), lr=config.LR)

# initialize a dictionary to store TRAINING history (keep track on training)
training_history = {"avg_train_loss": [], "train_accuracy": [], "IoU":[],"f1score":[], "avgDice":[]}

# # initialize a dictionary to store VALIDATION history (keep track on VALIDATION)
validation_history = {"avg_val_loss": [], "val_accuracy": [], "IoU_val":[], "f1score_val":[]}

# Using log="all" log histograms of parameter values in addition to gradients
wandb.watch(unet, log="all")

# Autocasting 
scaler = GradScaler()

# initialize best accuracy
best_accuracy = 0.0
print(f'''Training the network for {config.NUM_EPOCHS} epochs, with a batch size of {config.BATCH_SIZE}''') # try with logger

# loop = tqdm(range(config.NUM_EPOCHS))
iter_ = 0
for e in range(config.NUM_EPOCHS):
    trained = train(unet, train_dataloader, opt, lossFunc, epoch=e, scaler=scaler)
    validated = validation(unet, val_dataloader, lossFunc, epoch=e)
    
    # Save best model
    if validated['val_accuracy'][-1] > best_accuracy : # maybe add a minimum number of epochs as conditions
        utis.save_best_model(unet, BEST_MODEL, validated, e)
        best_accuracy = validation_history['val_accuracy'][-1]

wandb: Currently logged in as: vibferreira (use `wandb login --relogin` to force relogin)


Training the network for 5 epochs, with a batch size of 8


torch.Size([256, 256])
<class 'torch.Tensor'>


RuntimeError: result type Float can't be cast to the desired output type Byte

In [ ]:
# training_history

### Predictions

In [ ]:
# def save_predictions_as_imgs():
#     # test dataloader
#     pass

# # load the model 
# # Initialize our model
# unet = model.unet_model.to(config.DEVICE)
# path_model = '../best_model/best_model_epoch_4_acc_0.985_iou_0.003.pth' 
# unet.load_state_dict(torch.load(path_model))
# folder = '../saved_images' 

# y_hat_test = []
# y_true_test = []

# # switch off autograd
# with torch.no_grad():
#     # set the model in evaluation mode
#     unet.eval()

#     # loop over the validation set
#     loop = tqdm(test_dataloader, leave=False)

#     for idx, (x_test, y_test) in enumerate(loop):
#         # send the input to the device
#         (x_test, y_test)  = (x_test.to(config.DEVICE), y_test.to(config.DEVICE))

#         # Predictions
#         pred_test = unet(x_test)

#         # Assign appropriate class # ASK SEBASTIAN
#         pred_test = pred_test.sigmoid()
#         pred_test = (pred_test > 0.5).float()

#         # Storing predictions and true labels 
#         y_hat_test.append(pred_test)
#         y_true_test.append(y_test)

#         # metrics
#         utis.plot_comparison(x_test, pred_test, y_test)

#         # Save images
#         # save_image(pred_test, f"{folder}/pred_{idx}.png") 
#         # save_image(y_test, f"{folder}/y_true_{idx}.png")

# # # Stack and flatten for confusion matrix # GETTING SIZE ERROR AT THE MOMENT
# # torch.stack(y_hat_test).flatten()
# # torch.stack(y_true_test).flatten()

In [ ]:
# y_hat_test[0].detach().cpu()

In [ ]:
# _, ax = plt.subplots(1, 2, figsize=(15,5))
# ax[0].plot(training_history['avg_train_loss'], label= 'train')
# ax[0].plot(validation_history['avg_val_loss'], label='val')
# ax[0].set_title('Loss')
# ax[1].plot(training_history['train_accuracy'], label= 'train')
# ax[1].plot(validation_history['val_accuracy'], label='val')
# ax[1].set_title('Validation')
# ax[1].legend()
# plt.show()